## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-08-13-04-56 +0000,nyt,Must the Military Disobey Unlawful Orders? Pam...,https://www.nytimes.com/2025/12/08/us/politics...
1,2025-12-08-13-02-44 +0000,nyt,Syrians Celebrate Anniversary of Bashar al-Ass...,https://www.nytimes.com/2025/12/08/world/middl...
2,2025-12-08-13-02-09 +0000,bbc,More than 100 people killed in attack on hospi...,https://www.bbc.com/news/articles/c23e7kyl9myo...
3,2025-12-08-13-00-00 +0000,nypost,I developed a brain tumor the size of a lime a...,https://nypost.com/2025/12/08/health/long-isla...
4,2025-12-08-12-55-17 +0000,bbc,Police assess claims Reform overspent on Farag...,https://www.bbc.com/news/articles/cy4xrn0z0r0o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2422/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
58,trump,30
204,new,14
224,he,12
288,fire,11
112,deal,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
133,2025-12-08-02-40-46 +0000,wapo,Trump says he will be ‘involved’ in vetting Ne...,https://www.washingtonpost.com/business/2025/1...,72
233,2025-12-07-13-42-20 +0000,cbc,"Israel's Netanyahu says he will meet Trump, se...",https://www.cbc.ca/news/world/netanyahu-says-h...,66
164,2025-12-07-22-30-00 +0000,wsj,President Trump criticized as disloyal an indi...,https://www.wsj.com/politics/elections/trump-l...,63
63,2025-12-08-10-06-38 +0000,wapo,Trump peace deal falters as Thailand and Cambo...,https://www.washingtonpost.com/world/2025/12/0...,60
197,2025-12-07-19-42-14 +0000,cbc,Trump's new security strategy aligns with Russ...,https://www.cbc.ca/news/world/russia-trump-nat...,59


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
133,72,2025-12-08-02-40-46 +0000,wapo,Trump says he will be ‘involved’ in vetting Ne...,https://www.washingtonpost.com/business/2025/1...
68,36,2025-12-08-10-03-05 +0000,nyt,New Facial-Recognition Tech Could Let You Keep...,https://www.nytimes.com/2025/12/08/travel/faci...
139,35,2025-12-08-01-54-25 +0000,nypost,JPMorgan Chase’s Jamie Dimon claims AI will no...,https://nypost.com/2025/12/07/business/jamie-d...
214,32,2025-12-07-17-45-11 +0000,bbc,Hong Kong votes in election as city mourns dea...,https://www.bbc.com/news/articles/c3r7plnwr0zo...
81,29,2025-12-08-09-28-00 +0000,wsj,Japan Is Out Spending. Bond Markets Seem Nervo...,https://www.wsj.com/economy/japan-is-out-spend...
117,29,2025-12-08-04-15-18 +0000,nyt,China’s Trade Surplus Climbs Past $1 Trillion ...,https://www.nytimes.com/2025/12/07/business/ch...
26,28,2025-12-08-11-58-44 +0000,cbc,"Ukraine's Zelenskyy to meet with EU, NATO lead...",https://www.cbc.ca/news/world/ukraine-zelensky...
199,28,2025-12-07-19-29-38 +0000,nypost,Water leak at the Louvre damages hundreds of r...,https://nypost.com/2025/12/07/world-news/water...
190,28,2025-12-07-20-18-00 +0000,nypost,SL Green chief expects thriving NYC office ma...,https://nypost.com/2025/12/07/business/sl-gree...
209,27,2025-12-07-18-19-25 +0000,nypost,Beauty influencer was ‘subjected to violence’ ...,https://nypost.com/2025/12/07/world-news/stefa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
